In [ ]:
!pip install ultralytics opencv-python pandas ffmpeg-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 976.9/976.9 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import os

video_path = "/content/drone_footage.mp4"

if os.path.exists(video_path):
    print("✅ Video file exists!")
else:
    print("❌ Video file is missing! Please upload it.")


✅ Video file exists!


In [ ]:
#Extracting Frames from Video
import cv2
import os

output_folder = "/content/extracted_frames"
os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"✅ FPS: {fps}, Total Frames: {total_frames}")

frame_number = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_path = os.path.join(output_folder, f"frame_{frame_number:04d}.jpg")
    cv2.imwrite(frame_path, frame)
    frame_number += 1

cap.release()
print("✅ Frames extracted successfully!")


✅ FPS: 23.976023976023978, Total Frames: 642
✅ Frames extracted successfully!


In [ ]:
#Running Object Detection on Extracted Frames
from ultralytics import YOLO
import pandas as pd
import os

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

detection_results = []
frame_files = sorted(os.listdir("/content/extracted_frames"))

for frame_file in frame_files:
    frame_path = os.path.join("/content/extracted_frames", frame_file)
    results = model(frame_path)  # Run YOLOv8 on the frame

    for result in results:
        df = result.to_df()  # Use `.to_df()` instead of `.pandas().xyxy[0]`
        df["frame"] = frame_file  # Add frame name
        detection_results.append(df)

# Save detection results to CSV
df_all = pd.concat(detection_results, ignore_index=True)
df_all.to_csv("/content/detection_results.csv", index=False)

print("✅ Object detection completed and saved!")



image 1/1 /content/extracted_frames/frame_0000.jpg: 352x640 3 persons, 133.6ms
Speed: 2.7ms preprocess, 133.6ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /content/extracted_frames/frame_0001.jpg: 352x640 3 persons, 127.1ms
Speed: 2.7ms preprocess, 127.1ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /content/extracted_frames/frame_0002.jpg: 352x640 3 persons, 124.1ms
Speed: 2.8ms preprocess, 124.1ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /content/extracted_frames/frame_0003.jpg: 352x640 3 persons, 126.3ms
Speed: 2.7ms preprocess, 126.3ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /content/extracted_frames/frame_0004.jpg: 352x640 3 persons, 134.6ms
Speed: 3.9ms preprocess, 134.6ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /content/extracted_frames/frame_0005.jpg: 352x640 3 persons, 131.8ms
Speed: 3.3ms preprocess, 131.8

In [ ]:
#Filtering Alerts Based on Confidence & Objects
df_alerts = df_all[df_all["confidence"] > 0.5]  # Filter by confidence threshold

alert_classes = ["person", "car", "drone"]  # Define classes of interest
df_alerts = df_alerts[df_alerts["name"].isin(alert_classes)]

df_alerts.to_csv("/content/alert_detections.csv", index=False)
print(f"✅ {len(df_alerts)} alerts detected!")


✅ 1422 alerts detected!


In [ ]:
# Keeping only high-confidence alerts (e.g., confidence > 0.7)
df_alerts = df_alerts[df_alerts["confidence"] > 0.7]

# If "severity" exists, filter by highest severity levels
# df_alerts = df_alerts[df_alerts["severity"] >= 2]


In [ ]:

df_alerts = df_alerts.sort_values(by="frame")  # Sort by frame number
df_alerts = df_alerts.drop_duplicates(subset=["class"], keep="first")  # Keep only first detection of each object class in a frame


In [ ]:
# Extracting Alert Clips from the Video

import ffmpeg

alert_clips_folder = "/content/alert_clips"
os.makedirs(alert_clips_folder, exist_ok=True)

for index, row in df_alerts.iterrows():
    start_time = max(0, (row["frame"] / fps) - 2)  # 2 seconds before detection
    end_time = start_time + 5  # 5-second clip

    clip_path = os.path.join(alert_clips_folder, f"alert_clip_{index}.mp4")

    (
        ffmpeg
        .input(video_path, ss=start_time, to=end_time)
        .output(clip_path, vcodec="libx264", acodec="aac")
        .run(overwrite_output=True)
    )

    print(f"✅ Extracted: {clip_path}")





✅ Extracted: /content/alert_clips/alert_clip_0.mp4


In [ ]:
# Verify Clips & Playback
import glob

clips = glob.glob("/content/alert_clips/*.mp4")
if clips:
    print(f"✅ {len(clips)} alert clips generated!")
else:
    print("❌ No clips were generated. Check detection results.")


✅ 29 alert clips generated!


In [ ]:
#Create a Text File Listing the Clips
import os

clips_folder = "/content/alert_clips"
clip_files = sorted([f for f in os.listdir(clips_folder) if f.endswith(".mp4")])

# Ensure clips are found
if not clip_files:
    print("No clips found!")
else:
    # Create a file list for ffmpeg
    with open("file_list.txt", "w") as f:
        for clip in clip_files:
            f.write(f"file '{clips_folder}/{clip}'\n")

    print("File list created successfully!")


File list created successfully!


In [ ]:
!ffmpeg -f concat -safe 0 -i /content/alert_clips.txt -c copy /content/alert_clips/merged_alert_clips.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
ls -lh /content/alert_clips/


total 274M
-rw-r--r-- 1 root root 9.8M Mar 31 06:20 alert_clip_0.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 05:47 alert_clip_10.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 05:48 alert_clip_12.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 05:50 alert_clip_13.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 05:51 alert_clip_15.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 05:53 alert_clip_16.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 05:54 alert_clip_18.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 05:56 alert_clip_19.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 05:39 alert_clip_1.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 05:57 alert_clip_21.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 05:58 alert_clip_22.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 06:00 alert_clip_24.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 06:01 alert_clip_25.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 06:03 alert_clip_27.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 06:04 alert_clip_28.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 06:05 alert_clip_30.mp4
-rw-r--r-- 1 root root 9.8M Mar 31 06:07 alert_

In [ ]:
!cd /content/alert_clips/ && ls -1 *.mp4 | sed "s/^/file '/;s/$/'/" > /content/alert_clips.txt


In [ ]:
cat /content/alert_clips.txt


file 'alert_clip_0.mp4'
file 'alert_clip_10.mp4'
file 'alert_clip_12.mp4'
file 'alert_clip_13.mp4'
file 'alert_clip_15.mp4'
file 'alert_clip_16.mp4'
file 'alert_clip_18.mp4'
file 'alert_clip_19.mp4'
file 'alert_clip_1.mp4'
file 'alert_clip_21.mp4'
file 'alert_clip_22.mp4'
file 'alert_clip_24.mp4'
file 'alert_clip_25.mp4'
file 'alert_clip_27.mp4'
file 'alert_clip_28.mp4'
file 'alert_clip_30.mp4'
file 'alert_clip_31.mp4'
file 'alert_clip_33.mp4'
file 'alert_clip_34.mp4'
file 'alert_clip_36.mp4'
file 'alert_clip_37.mp4'
file 'alert_clip_39.mp4'
file 'alert_clip_3.mp4'
file 'alert_clip_40.mp4'
file 'alert_clip_41.mp4'
file 'alert_clip_4.mp4'
file 'alert_clip_6.mp4'
file 'alert_clip_7.mp4'
file 'alert_clip_9.mp4'


In [ ]:
!cd /content/alert_clips/ && ls -1 *.mp4 | sed "s|^|file '/content/alert_clips/|;s|$|'|" > /content/alert_clips.txt


In [ ]:
!ffmpeg -f concat -safe 0 -i /content/alert_clips.txt -c copy /content/alert_clips/merged_alert_clips.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
ls -lh /content/alert_clips/merged_alert_clips.mp4


-rw-r--r-- 1 root root 234M Mar 31 06:45 /content/alert_clips/merged_alert_clips.mp4


In [ ]:
from IPython.display import display, Video

video_path = "/content/alert_clips/merged_alert_clips.mp4"
display(Video(video_path, embed=True))



In [ ]:
#saving video to local system
from google.colab import files
files.download("/content/alert_clips/merged_alert_clips.mp4")
